In [ ]:
Есть потенциал для дорков

https://rivegauche.ru/robots.txt
https://rivegauche.ru/sitemap.xml
https://api.rivegauche.ru/
    
    
из xml можно составить финальный вариант базы и фиксировать заполненность относительно него
а так - заполнять динамически из того, что идет

остатки можно добить поиском, но убрав цифры и "-"
финальный таргет по списку url
попадание считается если часть урла без ссылок совпадает

In [ ]:
https://rivegauche.ru/sitemap2.xml - все товары
    
https://rivegauche.ru/sitemap3.xml - все категории
https://rivegauche.ru/sitemap4.xml - все бренды

In [ ]:
На мобильном сайте есть скрипт, который гасит сессию в случае несовпадения сертификатов
Работает как мера против проксей, которые расшифровывают https-трафик

В приложении защиты нету, перехватил прямой доступ к апишке приложения лол
Надо попробовать из приложения выцепить полные списки каталогов с айдишниками
Пока прямые урлы не раюотают

общий дизайн решения - реверс-инжениринг трафика приложения, сайт
при этом не дает расшифровывать ssl-трафик

In [ ]:
https://api.rivegauche.ru/rg/v1/newRG/products/forProductCodes
    Запрос в апи по рекомендательной модели, что-то в хидерах
    
https://api.rivegauche.ru/rg/v1/newRG/products/988889
    запрос в апи по конкретному наименованию, полная инфа
    
https://api.rivegauche.ru/rg/v1/newRG/carts/current/product/988889
    упрощенный запрос по конкретному наименованию, есть урл
    
https://api.rivegauche.ru/rg/v1/newRG/location/cities/city_4400/stores?ifEmptySendAll=true
    локации всех магазинов, хз зачем но прикольно
    
https://api.rivegauche.ru/rg/v1/newRG/products/ac-search?offset=0&size=20&st=%D0%BA%D1%80%D0%B5%D0%BC
    поиск по наименованию
    
https://api.rivegauche.ru/rg/v1/newRG/brands
    каталог ВСЕХ брендов

https://api.rivegauche.ru/rg/v1/newRG/products/search?brandCode=rg_brand_646&currentPage=0&pageSize=20
    каталог товаров по каждому бренду в отдельности
    
https://api.rivegauche.ru/rg/v1/newRG/catalog
    каталог товаров по категориям
    
https://api.rivegauche.ru/rg/v1/newRG/products/search?categoryCode=Perfumery_Woman&q=%3A&currentPage=0&pageSize=20
    каталог товаров по каждой категории по страницам
    
    
    
Чисто на всякий случай:
https://api.rivegauche.ru/rg/v1/newRG/products/autocomplete?st=%D0%BA%D1%80%D0%B5%D0%BC&brandsSize=4&categoriesSize=4&productsSizeForRender=6&productsSize=6
    скрипт автозаполнения
    
https://api.rivegauche.ru/rg/v1/newRG/carts/current?fields=FULL
    данные корзины

In [1]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                #every = int(size / 200)     # every 0.5%
                every = 1
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )
        
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)



In [2]:
import requests
import json
import pandas as pd 
import time
from datetime import datetime

headers = {
    'Content-Type': 'application/json', 
}

response = requests.get('https://rivegauche.ru/sitemap2.xml')

response

parsed_xml_list = response.text.replace('</loc>', '<loc>').split('<loc>')
filtered_parsed_list = []

for i in parsed_xml_list:
    # чекаем чтобы был дефис, чекаем чтобы были цифры, чекаем чтобы не было лишних категорий, чекаем чтобы в мапинге первым элементов была цифра
    if 'rivegauche.ru' in i:
        filtered_parsed_list.append(i)
        
len(filtered_parsed_list)

B:\python\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


32223

In [3]:
filtered_parsed_list[0:50] # нужно убрать в конце цифру если она двух или трехзначная

filtered_parsed_list_with_no_weights = []
for i in filtered_parsed_list:
    if i.split('-')[-1].isdigit() == True:
        filtered_parsed_list_with_no_weights.append( i.replace('-'+str(i.split('-')[-1]),'') )
        
filtered_parsed_list = list(set(filtered_parsed_list_with_no_weights))

In [4]:
# получаем бренды

headers = {
    'Content-Type': 'application/json', 
}

response = requests.get('https://api.rivegauche.ru/rg/v1/newRG/brands')

response

brands_json = json.loads(response.text)

len(brands_json)

556

In [5]:
# делаем каталог айди брендов

all_brands_id_catalog = []

for i in brands_json:
    all_brands_id_catalog.append(i['code'])
    
#all_brands_id_catalog

In [10]:
# сразу пишем финальный парсер
# попробую прямую итерацию без проксей

# важный момент - сделать отлавливание ошибок и трай-эксепт
# при первом прогоне ошибок парсинга не обнаружено, прогон успешен

all_products_from_list = []
acquired_sku_s = []

# в данном случае я не уверен в финальном списке товаров,
# так что логгинг прогресса идет через бренды

# выше описан план, в дальнейшем можно написать скрипт сверки с сайтмапом
# и в процентах по url учитывать его заполнение.
# А так - можно просто сравнить длину полученных данных
# с колвом записей из сайтмапа

for iter_brand_id in log_progress(all_brands_id_catalog):
    # итерируем все бренды
    
    headers = {
    'Content-Type': 'application/json', 
    }

    brand_id = iter_brand_id
    current_page = '0'

    response = requests.get('https://api.rivegauche.ru/rg/v1/newRG/products/search?brandCode=%s&currentPage=%s&pageSize=100'% (brand_id, current_page))

    temp_brand_products_json = json.loads(response.text)
    
    for page_id in range(0, temp_brand_products_json['pagination']['totalPages']):
        # в каждом бренде итерируем по всем его товарам, добавляем их в список
        
        if page_id != 0:
            # прогон нулевой был сделан вне этого цикла
            headers = {
            'Content-Type': 'application/json', 
            }

            brand_id = iter_brand_id
            current_page = str(page_id)

            response = requests.get('https://api.rivegauche.ru/rg/v1/newRG/products/search?brandCode=%s&currentPage=%s&pageSize=100'% (brand_id, current_page))

            temp_brand_products_json = json.loads(response.text)

        for iter_brand_info in temp_brand_products_json['results']:
            
            try:
                iter_brand_info['discountPrice']
            except Exception as e:
                if 'discountPrice' not in str(e):
                    print(e)
                    # logger.info("troubles with json: {%s}", e)
                iter_brand_info['discountPrice'] = iter_brand_info['price']
                
            cases_dict = iter_brand_info
            your_keys = ['code', 'name', 'url','stock', 'availableForPickup', 'manufacturer', 'price', 'volumePricesFlag', 'subtitle', 'prices', 
                         'brand', 'canAddToCart', 'deleted', 'adult', 'discountPrice', 'categoriesChain']
            temp_product_dict = { your_key: cases_dict[your_key] for your_key in your_keys }

            if temp_product_dict['code'] not in acquired_sku_s:
                acquired_sku_s.append(temp_product_dict['code'])
                all_products_from_list.append(temp_product_dict)
                
    # делаем таймаут между брендами
    time.sleep(1)

In [11]:
len(all_products_from_list)

18492

In [12]:
len(filtered_parsed_list)

3812

In [23]:
check_list = []

for i in all_products_from_list:
    check_list.append(str(i))

# все оригинальные
len(set(check_list))

18492

In [15]:
now = datetime.now()
print('finish:', now)

finish: 2022-11-18 02:45:43.492091


In [17]:
with open('data_rivegauche.json', 'w', encoding='utf-8') as f:
    json.dump(all_products_from_list, f, ensure_ascii=False, indent=4)

In [25]:
all_products_from_list[-1]

{'code': '867392',
 'name': '3Lab Perfect Neck Cream',
 'url': '/product/3lab-perfect-neck-cream',
 'stock': {'stockLevelStatus': 'outOfStock', 'stockLevel': 0},
 'availableForPickup': False,
 'manufacturer': '3LAB',
 'price': {'currencyIso': 'RUB',
  'value': 15382.0,
  'priceType': 'BUY',
  'formattedValue': '15\xa0382&nbsp;₽'},
 'volumePricesFlag': False,
 'subtitle': 'Увлажняющий крем для шеи, повышающий упругость кожи',
 'prices': [{'currencyIso': 'RUB',
   'value': 15382.0,
   'priceType': 'BUY',
   'formattedValue': '15\xa0382&nbsp;₽',
   'priceGroupCode': 'goldCardPriceGroup'},
  {'currencyIso': 'RUB',
   'value': 19484.0,
   'priceType': 'BUY',
   'formattedValue': '19\xa0484&nbsp;₽',
   'priceGroupCode': 'blueCardPriceGroup'},
  {'currencyIso': 'RUB',
   'value': 20510.0,
   'priceType': 'BUY',
   'formattedValue': '20\xa0510&nbsp;₽',
   'priceGroupCode': 'basePriceGroup'}],
 'brand': {'code': 'rg_brand_407',
  'name': '3LAB',
  'logo': {'url': '/medias6/?context=bWFzdGVyfGJy

In [ ]:
stop

In [ ]:
# в общем идея в том, что если в резалтах будет сотня, то тогда догружать некст страницу
headers = {
    'Content-Type': 'application/json', 
}

brand_id = 'rg_brand_675'
current_page = '0'

response = requests.get('https://api.rivegauche.ru/rg/v1/newRG/products/search?brandCode=%s&currentPage=%s&pageSize=100'% (brand_id, current_page))

temp_brand_products_json = json.loads(response.text)

In [ ]:
temp_brand_products_json

In [ ]:
temp_brand_products_json.keys()

In [ ]:
# количество страниц в запросе
temp_brand_products_json['pagination']['totalPages']


In [ ]:
temp_brand_products_json['results'][0]

In [ ]:
temp_brand_products_json['results'][0].keys()